# 02L - Embeddings, Vector Databases, and Search


In this lab, we will apply the text vectorization, search, and question answering workflow that you learned in the demo. The dataset we will use this time will be on talk titles and sessions from [Data + AI Summit 2023](https://www.databricks.com/dataaisummit/). 

![Dolly](https://files.training.databricks.com/images/llm/dolly_small.png)

### Learning Objectives
1. Learn how to use Chroma to store your embedding vectors and conduct similarity search
1. Use OpenAI GPT-3.5 to generate response to your prompt

`pip install chromadb==0.3.21 tiktoken==0.3.3`


In [4]:
from pathlib import Path

In [16]:
DA_paths_datasets = "../data"
assert Path(DA_paths_datasets).exists()


DA_paths_user_db = "../userdb"

In [22]:
from notebook_helper import (
    notebook_add_parent_dir_to_path,
)

notebook_add_parent_dir_to_path()

Added parent directory /Users/mjboothaus/code/github/mjboothaus/large-language-models-da to PYTHONPATH


In [23]:
from Includes.Test_Framework import dbTestQuestion2_1, dbTestQuestion2_2, dbTestQuestion2_3

Importing lab testing framework.


## Read data

In [10]:
!ls $DA_paths_datasets/dais

daistalks_23.parquet


In [11]:
import pandas as pd

dais_pdf = pd.read_parquet(f"{DA_paths_datasets}/dais/dais23_talks.parquet")
display(dais_pdf)

,Title,Abstract
0,AI in Healthcare,This session will explore the transformative i...
1,Natural Language Processing: From Theory to Pr...,Dive into the world of Natural Language Proces...
2,Reinforcement Learning in Autonomous Systems,Discover how reinforcement learning techniques...
3,Ethical Considerations in AI Development,"As AI continues to evolve, ethical considerati..."
4,Deep Learning for Computer Vision,Explore the world of deep learning and compute...
5,AI-powered Fraud Detection,Fraud detection is a critical challenge for or...
6,Generative Adversarial Networks: Creating Real...,Learn how generative adversarial networks (GAN...
7,Explainable AI: Interpreting Black Box Models,Interpretability is a key factor in deploying ...
8,AI for Financial Services,This session will showcase how AI is revolutio...
9,Reinventing Customer Experience with AI,Discover how AI is reshaping the customer expe...


In [12]:
dais_pdf["full_text"] = dais_pdf.apply(
    lambda row: f"""Title: {row["Title"]}
                Abstract:  {row["Abstract"]}""".strip(),
    axis=1,
)

print(dais_pdf.iloc[0]["full_text"])

texts = dais_pdf["full_text"].to_list()

Title: AI in Healthcare
                Abstract:  This session will explore the transformative impact of artificial intelligence in the healthcare industry. Artificial intelligence (AI) has the potential to revolutionize healthcare by improving diagnostics, enabling personalized treatment plans, and enhancing patient monitoring. Join us as we delve into the latest advancements in medical imaging analysis, predictive analytics for disease management, and AI-powered clinical decision support systems. Discover how AI algorithms can analyze vast amounts of patient data, identify patterns, and provide valuable insights for healthcare professionals. Learn about the challenges and opportunities of implementing AI in healthcare and the potential for improving patient outcomes and reducing healthcare costs.


In [13]:
dais_pdf

,Title,Abstract,full_text
0,AI in Healthcare,This session will explore the transformative i...,Title: AI in Healthcare\n Abstr...
1,Natural Language Processing: From Theory to Pr...,Dive into the world of Natural Language Proces...,Title: Natural Language Processing: From Theor...
2,Reinforcement Learning in Autonomous Systems,Discover how reinforcement learning techniques...,Title: Reinforcement Learning in Autonomous Sy...
3,Ethical Considerations in AI Development,"As AI continues to evolve, ethical considerati...",Title: Ethical Considerations in AI Developmen...
4,Deep Learning for Computer Vision,Explore the world of deep learning and compute...,Title: Deep Learning for Computer Vision\n ...
5,AI-powered Fraud Detection,Fraud detection is a critical challenge for or...,Title: AI-powered Fraud Detection\n ...
6,Generative Adversarial Networks: Creating Real...,Learn how generative adversarial networks (GAN...,Title: Generative Adversarial Networks: Creati...
7,Explainable AI: Interpreting Black Box Models,Interpretability is a key factor in deploying ...,Title: Explainable AI: Interpreting Black Box ...
8,AI for Financial Services,This session will showcase how AI is revolutio...,Title: AI for Financial Services\n ...
9,Reinventing Customer Experience with AI,Discover how AI is reshaping the customer expe...,Title: Reinventing Customer Experience with AI...


## Question 1

Set up Chroma and create collection


In [17]:
import chromadb
from chromadb.config import Settings

chroma_client = chromadb.Client(
    Settings(
        chroma_db_impl="duckdb+parquet",
        persist_directory=DA_paths_user_db,  # this is an optional argument. If you don't supply this, the data will be ephemeral
    )
)

Using embedded DuckDB with persistence: data will be stored in: ../userdb


Assign the value of `my_talks` to the `collection_name` variable.

In [19]:
collection_name = "my_talks"

In [20]:
# If you have created the collection before, you need to delete the collection first
if len(chroma_client.list_collections()) > 0 and collection_name in [chroma_client.list_collections()[0].name]:
    chroma_client.delete_collection(name=collection_name)
else:
    print(f"Creating collection: '{collection_name}'")
    talks_collection = chroma_client.create_collection(name=collection_name)

No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


Creating collection: 'my_talks'


/Users/mjboothaus/code/github/mjboothaus/large-language-models-da/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
dbTestQuestion2_1(collection_name)

PASSED: All tests passed for lesson2, question1
RESULTS RECORDED: Click `Submit` when all questions are completed to log the results.


## Question 2

[Add](https://docs.trychroma.com/reference/Collection#add) data to the collection. 

In [ ]:
# TODO
talks_collection.add(
    documents=<FILL_IN>,
    ids=<FILL_IN>
)


### Test your answer. DO NOT MODIFY THIS CELL.

In [25]:
dbTestQuestion2_2(talks_collection)

AssertionError: Test NOT passed: The collection should be non-empty.

## Question 3

[Query](https://docs.trychroma.com/reference/Collection#query) for relevant documents. If you are looking for talks related to language models, your query texts could be `language models`. 

In [27]:
import json

results = talks_collection.query(query_texts="language models", n_results=4)

print(json.dumps(results, indent=4))

NoIndexException: Index not found, please create an instance before querying

In [ ]:
# Test your answer. DO NOT MODIFY THIS CELL.

dbTestQuestion2_3(results)

## Question 4

Load a language model and create a [pipeline](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines).

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Pick a model from HuggingFace that can generate text
model_id = "<FILL_IN>"
tokenizer = AutoTokenizer.from_pretrained(model_id)
lm_model = AutoModelForCausalLM.from_pretrained(model_id)

pipe = pipeline(
    "<FILL_IN>",
    model=lm_model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    device_map="auto",
    handle_long_generation="hole",
)

In [ ]:
# Test your answer. DO NOT MODIFY THIS CELL.

dbTestQuestion2_4(pipe)

## Question 5

Prompt engineering for question answering


In [ ]:

# Come up with a question that you need the LLM assistant to help you with
# A sample question is "Help me find sessions related to XYZ" 
# Note: Your "XYZ" should be related to the query you passed in Question 3. 
question = "<FILL_IN>"

# Provide all returned similar documents from the cell above below
context = <FILL_IN>

# Feel free to be creative how you construct the prompt. You can use the demo notebook as a jumpstart reference.
# You can also provide more requirements in the text how you want the answers to look like.
# Example requirement: "Recommend top-5 relevant sessions for me to attend."
prompt_template = <FILL_IN>

In [ ]:
# Test your answer. DO NOT MODIFY THIS CELL.

dbTestQuestion2_5(question, context, prompt_template)

## Question 6 

Submit query for language model to generate response.

Hint: If you run into the error `index out of range in self`, make sure to check out this [documentation page](https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.TextGenerationPipeline.__call__.handle_long_generation).


In [ ]:
lm_response = pipe(<FILL_IN>)
print(lm_response[0]["generated_text"])

In [ ]:
# Test your answer. DO NOT MODIFY THIS CELL.

dbTestQuestion2_6(lm_response)

Notice that the output isn't exactly helpful. Head on to using OpenAI to try out GPT-3.5 instead! 

## OPTIONAL (Non-Graded): Use OpenAI models for Q/A

For this section to work, you need to generate an Open AI key. 

Steps:
1. You need to [create an account](https://platform.openai.com/signup) on OpenAI. 
2. Generate an OpenAI [API key here](https://platform.openai.com/account/api-keys). 

Note: OpenAI does not have a free option, but it gives you $5 as credit. Once you have exhausted your $5 credit, you will need to add your payment method. You will be [charged per token usage](https://openai.com/pricing). **IMPORTANT**: It's crucial that you keep your OpenAI API key to yourself. If others have access to your OpenAI key, they will be able to charge their usage to your account! 



In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "<FILL IN>"

import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

If you would like to estimate how much it would cost to use OpenAI, you can use `tiktoken` library from OpenAI to get the number of tokens from your prompt.

We will be using `gpt-3.5-turbo` since it's the most economical option at ($0.002/1k tokens), as of May 2023. GPT-4 charges $0.04/1k tokens. The following code block below is referenced from OpenAI's documentation on ["Managing tokens"](https://platform.openai.com/docs/guides/chat/managing-tokens).


In [ ]:
import tiktoken

price_token = 0.002
encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")
cost_to_run = len(encoder.encode(prompt_template)) / 1000 * price_token
print(f"It would take roughly ${round(cost_to_run, 5)} to run this prompt")

We won't have to create a new vector database again. We can just send our `context` from above to OpenAI. We will use their chat completion API to interact with `GPT-3.5-turbo`. You can refer to their [documentation here](https://platform.openai.com/docs/guides/chat).

Something interesting is that OpenAI models use the system message to help their assistant to be more accurate. From OpenAI's [docs](https://platform.openai.com/docs/guides/chat/introduction):

> Future models will be trained to pay stronger attention to system messages. The system message helps set the behavior of the assistant.


In [ ]:
gpt35_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": <FILL_IN>},
    ],
    temperature=0, # 0 makes outputs deterministic; The closer the value is to 1, the more random the outputs are for each time you re-run.
)

print(gpt35_response.choices[0]["message"]["content"])


from IPython.display import Markdown

Markdown(gpt35_response.choices[0]["message"]["content"])


We can also check how many tokens OpenAI has used


In [ ]:
gpt35_response["usage"]["total_tokens"]


The results are noticeably much better compared to when using Hugging Face's GPT-2! It didn't get stuck in the text generation, but the sessions recommended are not all relevant to pandas either. You can further do more prompt engineering to get better results.